In [2]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from sklearn.utils import check_array
import xgboost as xgb
import model_dispatcher
import config

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.offline as pyoff
import plotly.graph_objs as go
import plotly.express as px
import joblib

ModuleNotFoundError: No module named 'model_dispatcher'

In [38]:
# read the csv file
file_path = "D:\Murtaza\machine_learning_projects\mlframework\PrognosixAG-CaseStudy\input/"
df = pd.read_csv(file_path + "tomatoesAndFeatures.csv", delimiter=';', parse_dates=['date'])
df = df.sort_values(by="date")

In [39]:
fig = px.line(df, x='date', y='salesAmount', title='Amount of Sales')
fig.update_xaxes(rangeslider_visible=True)
fig.show()

In [40]:
# create a new dataframe to model the difference 
df_diff = df.copy()

# add previous sales to the next row
df_diff['prev_sales'] = df_diff['salesAmount'].shift(1)

# drop the null values and calculate the difference
df_diff = df_diff.dropna()
df_diff['diff'] = (df_diff['salesAmount'] - df_diff['prev_sales'])

df_diff.head(10)

,date,salesAmount,promotionYes,bridgeDayYes,publicHolidayYes,seasonCode,seasonName,calendarWeek,newSnowCode,newSnowDescription,...,rainDescription,sunshineCode,sunShineDescription,temperaturCode,temperatureDescription,shopsClosedYes,weekdayCode,weekDayDescription,prev_sales,diff
1,2012-11-02,6531,0,0,0,3,Herbst,44,0,kein Schnee,...,feuchter als Mittel,4,mehr als Mittel,5,viel waermer als Mittel,0,6,Freitag,4574.0,1957.0
2,2012-11-03,7291,0,0,0,3,Herbst,44,0,kein Schnee,...,Mittel,5,viel mehr als Mittel,5,viel waermer als Mittel,0,7,Samstag,6531.0,760.0
3,2012-11-04,611,0,0,0,3,Herbst,44,0,kein Schnee,...,viel feuchter als Mittel,4,mehr als Mittel,5,viel waermer als Mittel,1,1,Sonntag,7291.0,-6680.0
4,2012-11-05,4732,0,0,0,3,Herbst,45,0,kein Schnee,...,viel feuchter als Mittel,4,mehr als Mittel,4,waermer als Mittel,0,2,Montag,611.0,4121.0
5,2012-11-06,4270,0,0,0,3,Herbst,45,0,kein Schnee,...,feuchter als Mittel,3,Mittel,2,kaelter als Mittel,0,3,Dienstag,4732.0,-462.0
6,2012-11-07,5182,0,0,0,3,Herbst,45,0,kein Schnee,...,viel trockener als Mittel,3,Mittel,2,kaelter als Mittel,0,4,Mittwoch,4270.0,912.0
7,2012-11-08,4783,0,0,0,3,Herbst,45,0,kein Schnee,...,viel trockener als Mittel,3,Mittel,2,kaelter als Mittel,0,5,Donnerstag,5182.0,-399.0
8,2012-11-09,7285,0,0,0,3,Herbst,45,0,kein Schnee,...,feuchter als Mittel,5,viel mehr als Mittel,3,Mittel,0,6,Freitag,4783.0,2502.0
9,2012-11-10,8423,0,0,0,3,Herbst,45,0,kein Schnee,...,feuchter als Mittel,1,viel weniger als Mittel,5,viel waermer als Mittel,0,7,Samstag,7285.0,1138.0
10,2012-11-11,653,0,0,0,3,Herbst,45,0,kein Schnee,...,viel feuchter als Mittel,1,viel weniger als Mittel,5,viel waermer als Mittel,1,1,Sonntag,8423.0,-7770.0


In [41]:
fig = px.line(df_diff, x='date', y='diff', title='Monthly Sales Diff')
fig.update_xaxes(rangeslider_visible=True)
fig.show()

The data is not completely stationery but tis can be used to build our feature set
We need to use the previous monthly sales data to forecast the next ones. 

In [36]:
df_new.head()

,salesAmount,promotionYes,bridgeDayYes,publicHolidayYes,seasonCode,calendarWeek,newSnowCode,rainCode,sunshineCode,temperaturCode,shopsClosedYes,weekdayCode
0,4574,0,0,0,3,44,0,4,5,4,0,5
1,6531,0,0,0,3,44,0,4,4,5,0,6
2,7291,0,0,0,3,44,0,3,5,5,0,7
3,611,0,0,0,3,44,0,5,4,5,1,1
4,4732,0,0,0,3,45,0,5,4,4,0,2


In [42]:
# dropping redundant columns

columns = [
    'date',
    'seasonName', 
    'newSnowDescription',
    'rainDescription',
    'sunShineDescription',
    'temperatureDescription',
    'weekDayDescription',
]

df_new = df.drop(columns, axis=1)

In [43]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1769 entries, 0 to 1768
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   salesAmount       1769 non-null   int64
 1   promotionYes      1769 non-null   int64
 2   bridgeDayYes      1769 non-null   int64
 3   publicHolidayYes  1769 non-null   int64
 4   seasonCode        1769 non-null   int64
 5   calendarWeek      1769 non-null   int64
 6   newSnowCode       1769 non-null   int64
 7   rainCode          1769 non-null   int64
 8   sunshineCode      1769 non-null   int64
 9   temperaturCode    1769 non-null   int64
 10  shopsClosedYes    1769 non-null   int64
 11  weekdayCode       1769 non-null   int64
dtypes: int64(12)
memory usage: 179.7 KB


In [51]:
# split train and test data with a ratio of 0.8:0.2
df_train, df_test = df_new[0:-1415], df_new[-1415:]

xtrain = df_train.drop(['salesAmount'], axis=1).values
ytrain = df_train.drop([f for f in df_train.columns if f not in ('salesAmount')], axis=1).values

xtest = df_test.drop(['salesAmount'], axis=1).values
ytest = df_test.drop([f for f in df_test.columns if f not in ('salesAmount')], axis=1).values


In [55]:
# Apply MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1,1))
scaler = scaler.fit(xtrain)

# reshape training set 
xtrain = xtrain.reshape(xtrain.shape[0], xtrain.shape[1])
xtrain_scaled = scaler.transform(xtrain)

# reshape test set 
xtest = xtest.reshape(xtest.shape[0], xtest.shape[1])
xtest_scaled = scaler.transform(xtest)

In [288]:
# transform a time series dataset into a supervised learning dataset
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    """
    :param data: dataframe
    :param n_in: number of input to be given to the model
    :param n_out = number of output from the model
    :param dropna: to drop the nan values
    :return: the aggregated rows 
    """
    n_vars = data.shape[1]
    df = pd.DataFrame(data)
    cols = list()
	
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
	
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
	
    # put it all together
    agg = pd.concat(cols, axis=1)
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg.values

In [289]:
# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
    """
    :param data: input dataframe
    :param data: input test split
    """
    return data[:-n_test, :], data[-n_test:, :]


In [295]:
# fit an random forest model and make a one step prediction
def random_forest_forecast(train, testX):
	# transform list into array
	train = np.asarray(train)
	# split into input and output columns
	trainX, trainy = train[:, 1:], train[:, 0]
	# fit model
	model = ensemble.RandomForestRegressor(n_estimators=1000)
	model.fit(trainX, trainy)
	# make a one-step prediction
	yhat = model.predict([testX])
	return yhat[0]

In [296]:
# walk-forward validation for univariate data
def walk_forward_validation(data, n_test):
    """
    :param data: dataframe
    :param n_test: split amount for test data
    """
    predictions = list()

	# split dataset
    train, test = train_test_split(data, n_test)

    # seed history with training dataset
    history = [x for x in train]
    counter = 0
    # step over each time-step in the test set
    for i in range(len(test)):
		# split test row into input and output columns
        testX, testy = test[i, 1:], test[i, 0]
        # fit model on history and make a prediction
        yhat = random_forest_forecast(history, testX)
		# store forecast in list of predictions
        predictions.append(yhat)
		# add actual observation to history for the next loop
        history.append(test[i])
		# summarize progress
        print('>expected=%.1f, predicted=%.1f' % (testy, yhat))
	# estimate prediction error
    error = mean_absolute_error(test[:, 0], predictions)
    return error, test[:, 0], predictions

In [297]:
# load the dataset
df_ = pd.read_csv(file_path + "tomatoesAndFeatures.csv", delimiter=';', parse_dates=['date'])
df_ = df_.sort_values(by="date")
df_ = df_.drop(columns, axis=1)

values = df_.values

# transform the time series data into supervised learning
data = series_to_supervised(values, n_in=6)

# evaluate
mae, y, yhat = walk_forward_validation(data, 246)
print('MAE: %.3f' % mae)

# plot expected vs predicted
plt.plot(y, label='Expected')
plt.plot(yhat, label='Predicted')
plt.legend()
plt.show()

>expected=14036.0, predicted=12146.1
>expected=11169.0, predicted=12248.8
>expected=11314.0, predicted=13086.5
>expected=16986.0, predicted=16191.3
>expected=17065.0, predicted=3385.9
>expected=1329.0, predicted=2112.4
>expected=7219.0, predicted=14085.6
>expected=6479.0, predicted=4728.0
>expected=5203.0, predicted=5167.5
>expected=4888.0, predicted=5972.3
>expected=7782.0, predicted=9711.2
>expected=12439.0, predicted=12086.2
>expected=1570.0, predicted=1454.7
>expected=8079.0, predicted=9053.7
>expected=13831.0, predicted=16997.3
>expected=15893.0, predicted=15266.1
>expected=14125.0, predicted=14219.8
>expected=18756.0, predicted=18984.9
>expected=26249.0, predicted=26342.9
>expected=2366.0, predicted=2452.1
>expected=15063.0, predicted=15868.9
>expected=7021.0, predicted=6863.7
>expected=6942.0, predicted=6948.9
>expected=6899.0, predicted=7166.1
>expected=10241.0, predicted=9648.1
>expected=13045.0, predicted=13148.1
>expected=1544.0, predicted=1461.3
>expected=8261.0, predicted=

NameError: name 'mean_absolute_error' is not defined

In [287]:
print(data[1])
print(data[2])

[6.531e+03 0.000e+00 0.000e+00 0.000e+00 3.000e+00 4.400e+01 0.000e+00
 4.000e+00 4.000e+00 5.000e+00 0.000e+00 6.000e+00 7.291e+03 0.000e+00
 0.000e+00 0.000e+00 3.000e+00 4.400e+01 0.000e+00 3.000e+00 5.000e+00
 5.000e+00 0.000e+00 7.000e+00 6.110e+02 0.000e+00 0.000e+00 0.000e+00
 3.000e+00 4.400e+01 0.000e+00 5.000e+00 4.000e+00 5.000e+00 1.000e+00
 1.000e+00 4.732e+03 0.000e+00 0.000e+00 0.000e+00 3.000e+00 4.500e+01
 0.000e+00 5.000e+00 4.000e+00 4.000e+00 0.000e+00 2.000e+00 4.270e+03
 0.000e+00 0.000e+00 0.000e+00 3.000e+00 4.500e+01 0.000e+00 4.000e+00
 3.000e+00 2.000e+00 0.000e+00 3.000e+00 5.182e+03 0.000e+00 0.000e+00
 0.000e+00 3.000e+00 4.500e+01 0.000e+00 1.000e+00 3.000e+00 2.000e+00
 0.000e+00 4.000e+00 4.783e+03 0.000e+00 0.000e+00 0.000e+00 3.000e+00
 4.500e+01 0.000e+00 1.000e+00 3.000e+00 2.000e+00 0.000e+00 5.000e+00]
[7.291e+03 0.000e+00 0.000e+00 0.000e+00 3.000e+00 4.400e+01 0.000e+00
 3.000e+00 5.000e+00 5.000e+00 0.000e+00 7.000e+00 6.110e+02 0.000e+00
 0.00

In [132]:
tr, t = train_test_split(data, 247)

In [1]:
from sklearn.metrics import mean_absolute_percentage_error

ImportError: cannot import name 'mean_absolute_percentage_error' from 'sklearn.metrics' (D:\downloads\Programs\envs\py37\lib\site-packages\sklearn\metrics\__init__.py)

In [ ]:
y_true = [3, -0.5, 2, 7]
y_pred = [2.5, 0.0, 2, 8]
mean_absolute_percentage_error(y_true, y_pred)

y_true = [[0.5, 1], [-1, 1], [7, -6]]
y_pred = [[0, 2], [-1, 2], [8, -5]]
mean_absolute_percentage_error(y_true, y_pred)

mean_absolute_percentage_error(y_true, y_pred, multioutput=[0.3, 0.7])
